# Summary of Notation
---


(full notation in Appendix A)

> Whenever you see the word "value", think "action's Expected Value of reward" (AEVOR). This is usually a $P(win)$ or (in the bandit case) probability-weighted-avg $R_t$, given that that action is taken. It can be either a "True" value (i.e. the population mean of the action's distribution) or an estimate (i.e. sample statistic) of it.

| Symbol | Meaning |
|--------|---------|
| $\Pr\{X = x\}$ | Probability that random variable $X$ takes value $x$ |
| $X \sim p$ | $X$ is drawn from distribution $p$ |
| $\mathbb{E}[X]$ | Expectation of random variable $X$ |
| $\arg\max_a f(a)$ | Value of $a$ that maximizes $f(a)$ |
| $\alpha$, $\beta$, $\epsilon$ | Step-size, decay-rate, and exploration parameters |
| $s, s'$ | States |
| $a$ | Action |
| $r$ | Reward |
| $S, A(s), R$ | Set of states, available actions, rewards |
| $t, T$ | Discrete time step, final step |
| $S_t, A_t, R_t$ | State, action, and reward at time $t$ |
| $\pi$ | Policy (decision-making rule) |
| $\pi(s)$ | action taken in state $s$ under _deterministic_ $\pi$ |
| $\pi(a \mid s)$ | Probability of taking action $a$ in state $s$ under _stochastic_ $\pi$ |
| $G_t$ | Return from time $t$ |
| $h$ | horizon (the timestep one looks up to in a forward view) |
| $v_\pi(s)$ | Value of state $s$ under policy $\pi$ |
| $q_\pi(s, a)$ | Value of state-action pair $(s,a)$ under $\pi$ |
| $p(s', r \mid s, a)$ | Transition dynamics |
| $\hat{v}(s, \mathbf{w})$ | Approximate value function using weight vector $\mathbf{w}$ |
| $\delta_t$ | Temporal-difference error at time $t$ |

# Part I: Tabular Solution Methods
---

<img src="../img/6.png" alt="tabularsolutionmethods" width="40%"/>

# Chapter 7: n-step Bootstrapping
---

**Chapter Overview**: This chapter bridges Monte Carlo (MC) methods (Chapter 5) and one-step temporal-difference (TD) methods (Chapter 6) by introducing n-step TD methods. These generalize both approaches, allowing a smooth shift between them based on task needs. n-step methods form a spectrum: MC at one end (full episode backups) and one-step TD at the other (single-step backups). Intermediate n often performs best, balancing bias-variance trade-offs.

**Key Insight**: Neither pure MC (high variance, low bias) nor one-step TD (low variance, higher bias) is always optimal. n-step methods tune the bootstrapping amount for better performance.

**Sidenote**: Bootstrapping means updating estimates based on other estimates (like TD), reducing variance but introducing bias if estimates are poor. MC avoids bootstrapping by waiting for actual returns, but suffers from high variance in long episodes.

---

## 7.1 n-step TD Prediction

**Core Idea**: Estimate value function $v_\pi$ using n-step updates—look ahead n steps, then bootstrap from the estimated value at step n.

**Spectrum of Methods**:
- One-step TD: Update based on immediate reward + bootstrapped next state value.
- MC: Update based on all rewards until episode end (no bootstrapping).
- n-step TD: Update based on n rewards + bootstrapped value at n steps ahead.

![Backup Diagrams for n-step Methods](../img/fig7_1.png)

**n-step Return** (target for update, _Equation 7.1_):
$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n V_{t+n-1}(S_{t+n})$$
- For $t + n < T$; if $t + n \geq T$, $G_{t:t+n} = G_t$ (full return).
- Components:
  - $R_{t+1}$ to $R_{t+n}$: Actual rewards over n steps.
  - $\gamma^n V_{t+n-1}(S_{t+n})$: Bootstrapped estimate correcting for truncation.
- Intuition: Approximation to full return $G_t$; truncates after n steps and corrects with current value estimate.

**Update Rule** (_Equation 7.2_):
$$V_{t+n}(S_t) \doteq V_{t+n-1}(S_t) + \alpha [G_{t:t+n} - V_{t+n-1}(S_t)]$$
- For $0 \leq t < T$; values unchanged for unvisited states.
- $\alpha$: Step size.
- Intuition: Moves $V(S_t)$ toward the n-step return target.

**🏆 FOUNDATIONAL FORMULA**: The n-step return blends sampling (actual rewards) and bootstrapping, central to unifying TD and MC.

**Algorithm**: Pseudocode for n-step TD (state-value prediction under policy $\pi$).
- Initialize $V(s)$ arbitrarily.
- For each episode: Store states/rewards mod $n+1$; compute $G$ and update after n steps (or at end).

**Error Reduction Property**: n-step returns reduce error relative to true expected value more than shorter returns (proven for fixed $V$).

**Example: Random Walk** (19 states, undiscounted, rewards -1/0/+1 at ends).
- Intermediate n (e.g., 4-8) outperforms extremes for various $\alpha$.

![Performance on Random Walk](../img/fig7_2.png)

**Sidenote**: Larger tasks favor intermediate n as variance grows with episode length in MC, while one-step TD bootstraps too aggressively early on.

---

## 7.2 n-step Sarsa

**Core Idea**: Extend n-step to on-policy control for $q_\pi$ estimation.

**Backup Diagrams**:

![Backup Diagrams for n-step Action-Value Methods](../img/fig7_3.png)

**n-step Return for Action Values** (_Equation 7.4_):
$$G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n Q_{t+n-1}(S_{t+n}, A_{t+n})$$
- $t + n < T$; else $G_{t:t+n} = G_t$.

**Update Rule** (_Equation 7.5_):
$$Q_{t+n}(S_t, A_t) \doteq Q_{t+n-1}(S_t, A_t) + \alpha [G_{t:t+n} - Q_{t+n-1}(S_t, A_t)]$$

**Algorithm**: Pseudocode for n-step Sarsa.
- Similar to n-step TD but for $Q(s,a)$; actions from $\epsilon$-greedy policy.
- Speeds policy improvement by propagating rewards faster (e.g., cliff-walking example).

**n-step Expected Sarsa**: Uses expected value over actions at step n (_Equation 7.7_):
$$G_{t:t+n} = R_{t+1} + \cdots + \gamma^{n-1} R_{t+n} + \gamma^n \bar{V}_{t+n-1}(S_{t+n})$$
- $\bar{V}_t(s) = \sum_a \pi(a|s) Q_t(s,a)$.
- Often better than Sarsa due to lower variance.

**Sidenote**: Expected Sarsa generalizes Q-learning (one-step case) to multi-step.

---

## 7.3 n-step Off-policy Learning

**Core Idea**: Learn target policy $\pi$ from behavior policy $b$ data using importance sampling (IS).

**Importance Sampling Ratio** (_Equation 7.8_):
$$\rho_{t:h} = \prod_{k=t}^{h} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}$$
- Corrects for action selection differences.

**Off-policy n-step TD Update** (_Equation 7.9_):
$$V_{t+n}(S_t) \doteq V_{t+n-1}(S_t) + \alpha \rho_{t+1:t+n} [G_{t:t+n} - V_{t+n-1}(S_t)]$$
- Generalizes on-policy (ratio=1).

**Off-policy n-step Sarsa** (_Equation 7.10_):
$$Q_{t+n}(S_t, A_t) \doteq Q_{t+n-1}(S_t, A_t) + \alpha \rho_{t+1:t+n} [G_{t:t+n} - Q_{t+n-1}(S_t, A_t)]$$
- Ratio starts at $t+1$ (action $A_t$ already taken).

**Algorithm**: Pseudocode for off-policy n-step Sarsa.

**Sidenote**: IS can cause high variance if $\pi$ and $b$ differ much; zero ratio ignores bad samples, high ratio amplifies good ones.

---

## 7.4 *Per-decision Methods with Control Variates

**Core Idea**: Refine off-policy with per-decision IS and control variates (CV) for lower variance.

**Recursive n-step Return** (_Equation 7.12_):
$$G_{t:h} = R_{t+1} + \gamma G_{t+1:h}$$

**Off-policy Version with CV** (_Equation 7.13_):
$$G_{t:h} = R_{t+1} + \gamma \rho_t (G_{t+1:h} - Q_t(S_{t+1}, A_{t+1})) + \gamma \bar{V}_t(S_{t+1})$$
- CV term: Doesn't change expectation but reduces variance by ignoring unlikely branches.
- Intuition: If $\rho_t=0$, target = immediate reward + bootstrapped value (ignore sample).

**For Action Values** (_Equation 7.14_):
$$G_{t:h} = R_{t+1} + \gamma (\rho_{t+1} G_{t+1:h} + (1 - \rho_{t+1}) Q_t(S_{t+1}, A_{t+1})) + \gamma (1 - \rho_{t+1}) (\bar{V}_t(S_{t+1}) - Q_t(S_{t+1}, A_{t+1}))$$
- More complex; first action not importance-sampled.

**Sidenote**: CV exploits that unlikely actions under $\pi$ shouldn't bias updates; expected update unchanged.

---

## 7.5 Off-policy Learning Without Importance Sampling: The n-step Tree Backup Algorithm

**Core Idea**: Multi-step generalization of Expected Sarsa/Q-learning; branches over all actions (no IS).

**Backup Diagram**: Tree structure—sample spine, expected branches off states.

**n-step Return** (_Equation 7.15_ for 1-step, general recursive _Equation 7.17_):
$$G_{t:t+n} = R_{t+1} + \gamma \sum_{a \neq A_{t+1}} \pi(a|S_{t+1}) Q_{t+n-1}(S_{t+1}, a) + \gamma \pi(A_{t+1}|S_{t+1}) G_{t+1:t+n}$$
- Weights by $\pi$; samples only taken action.

**Update**: Same as Expected Sarsa but with tree return.

**Algorithm**: Pseudocode for n-step Tree Backup.

**Sidenote**: Avoids IS variance; effective when policies differ, but bootstraps less if $\pi(A_t|S_t)$ small.

---

## 7.6 *A Unifying Algorithm: n-step $Q(\sigma)$

**Core Idea**: Generalizes previous methods with per-step sampling degree $\sigma_k \in [0,1]$.

**Backup Diagrams**:

![Backup Diagrams for Q(sigma)](../img/fig7_5.png)

**n-step Return** (_Equation 7.18_):
$$G_{t:h} = R_{t+1} + \gamma [\sigma_{t+1} \rho_{t+1} (G_{t+1:h} - Q_{t}(S_{t+1}, A_{t+1})) + (1 - \sigma_{t+1} \rho_{t+1}) \bar{V}_{t}(S_{t+1})] + \gamma \sigma_{t+1} \rho_{t+1} Q_{t}(S_{t+1}, A_{t+1})$$
- $\sigma=1$: Full sampling (like Sarsa with IS).
- $\sigma=0$: Expectation (like Tree Backup).
- Intuition: Blends sampling (high variance, low bias) and expectation flexibly.

**Algorithm**: Pseudocode for off-policy n-step $Q(\sigma)$.

**🏆 FOUNDATIONAL**: Unifies on/off-policy, TD/MC, Sarsa/Q-learning; adaptable via $\sigma$.

---

## 7.7 Summary

**Key Concepts**:
- n-step methods: Tune bootstrapping degree for optimal bias-variance.
- On-policy: n-step TD/Sarsa/Expected Sarsa.
- Off-policy: IS-based (simple but high variance), Tree Backup (no IS), $Q(\sigma)$ (unifying).
- Advantages: Faster propagation, better than extremes.
- Trade-offs: More computation/memory than one-step; variance issues in off-policy.

**Fundamental Takeaways**:
1. Multi-step bootstrapping accelerates learning by balancing immediacy and completeness.
2. Off-policy needs correction (IS or trees) for policy mismatch.
3. $Q(\sigma)$: Flexible framework for future algorithms.

---

## Exercises

### Exercise 7.1 Solution
**Q**: In learning curves such as those shown in Figure 7.2, asymptote is offset slightly above 0 due to the off-by-0.5-step problem (random walk starts in center state 10, true values range 1/20 to 19/20). Can you explain why? What would the asymptotic error be if $\gamma=0.9$?

**A**: 
- Asymptotic RMS error isn't zero because learned values approximate $v_\pi$, but true values are offset (e.g., from center, expected return is average of left/right paths).
- With $\gamma=0.9$, error scales by discount; asymptotic error would be $0.5 \times (1 - \gamma) = 0.05$ (general undiscounted bias persists, but discounted reduces effective offset).

### Exercise 7.2 Solution
**Q**: Why does off-policy MC control require $\pi$ is greedy wrt $Q$, while on-policy MC does not?

**A**: Off-policy separates behavior ($b$, exploratory) and target ($\pi$, greedy). Greedy $\pi$ ensures optimality; on-policy uses single improving policy, converging to optimal without explicit greediness.

### Exercise 7.3 Solution
**Q**: Why larger random walk (19 vs 5 states)? Effect of smaller walk or left reward -1 vs 0 on best n?

**A**: 
- Larger: Amplifies variance in MC (longer episodes), shifts optimum n rightward.
- Smaller: Favors larger n (closer to MC).
- -1 left: Increases variance asymmetry; may favor slightly smaller n for stability.

### Exercise 7.4 Solution
**Q**: Prove n-step Sarsa return equals novel TD error form.

**A**:
Start with $G_{t:t+n} = Q_{t-1}(S_t, A_t) + \sum_{k=t}^{t+n-1} \gamma^{k-t} [R_{k+1} + \gamma Q_k(S_{k+1}, A_{k+1}) - Q_{k-1}(S_k, A_k)]$.
- By induction: Base $n=1$ holds; assume for n-1, expand recursively.

### Exercise 7.5 Solution
**Q**: Pseudocode for off-policy state-value prediction with CV.

**A**:
```python
# Off-policy n-step TD with per-decision IS and CV for V ≈ v_π
# Input: policies π, b; α; n
# Initialize V(s) arbitrarily
# For each episode:
#   Generate episode using b: S0, R1, S1, ..., RT
#   For t = 0 to T-n:
#       G = V(St+n)  # or adjust if terminal
#       For k = t+n-1 downto t:
#           ρ = π(Ak|Sk) / b(Ak|Sk)
#           G = R_{k+1} + γ * ρ * (G - V(S_{k+1})) + γ * V(S_{k+1})
#       V(St) += α * (G - V(St))
```

### Exercise 7.6 Solution
Q: Why per-decision methods better than ordinary IS for off-policy MC?

A: Per-decision applies IS only to relevant steps, adding CV to reduce variance (ignores low-prob branches without zeroing entire return).

### Exercise 7.7 Solution
Q: Prove per-decision off-policy return expectation equals on-policy return.

A: By induction on horizon h:

Base: $E[G_{h-1:h}] = E[R_h] = v_π(S_{h-1})$.
Assume for t+1:h; then $E[G_{t:h}] = E[R_{t+1} + γ ρ_t G_{t+1:h} + γ (1 - ρ_t) V(S_{t+1})] = ... = v_π(S_t)$, since $E[ρ_t]=1$, CV expectation 0.

# Appendix A: Full Notation
---

### General Notation

> Capital letters are used for random variables.

> Lower case letters are used for the values of random variables and for scalar functions.

> Quantities that are required to be real-valued vectors are written in bold and in lower case (even if random variables). 

> Matrices are bold capitals.

| Symbol | Meaning |
|--------|---------|
| **Standard Operators** |  |
| $\doteq$ | Equality relationship that is true by definition |
| $\approx$ | Approximately equal |
| $\propto$ | Proportional to |
| $\Pr\{X = x\}$ | Probability that random variable $X$ takes value $x$ |
| $X \sim p$ | $X$ drawn from distribution $p$ |
| $\mathbb{E}[X]$ | Expectation of random variable $X$ |
| $\arg\max_a f(a)$ | Value of $a$ that maximizes $f(a)$ |
| $\ln x$ | Natural logarithm of $x$ |
| $e^x$, $\exp(x)$ | The base of the natural logarithm, $e \approx 2.71828$, carried to power $x$ |
| $\mathbb{R}$ | Set of real numbers |
| $f: \mathcal{X} \to \mathcal{Y}$ | Function from elements of set $\mathcal{X}$ to elements of set $\mathcal{Y}$ |
| $\leftarrow$ | Assignment |
| $(a, b]$ | Real interval between $a$ and $b$ including $b$ but not $a$ |
| **Standard RL Algorithm Parameters** |  |
| $\epsilon$ | Probability of taking a random action in an $\epsilon$-greedy policy |
| $\alpha, \beta$ | Step-size parameters |
| $\gamma$ | Discount-rate parameter |
| $\lambda$ | Decay-rate parameter for eligibility traces |
| $\mathbf{1}_\text{predicate}$ | Indicator function (1 if predicate is true, else 0) |

---

### Multi-Armed Bandit Notation

| Symbol | Meaning |
|--------|---------|
| $k$ | Number of actions (arms) |
| $t$ | Discrete time step or play number |
| $q_*(a)$ | True value (expected reward) of action $a$ |
| $Q_t(a)$ | Estimate at time $t$ of $q_*(a)$ |
| $N_t(a)$ | Number of times action $a$ has been selected up to time $t$ |
| $H_t(a)$ | Learned preference for selecting action $a$ at time $t$ |
| $\pi_t(a)$ | Probability of selecting action $a$ at time $t$ |
| $\bar{R}_t$ | Estimate at time $t$ of the expected reward given $\pi_t$ |

---

### Markov Decision Process Notation

| Symbol | Meaning |
|--------|---------|
| **State & Action Sets** |  |
| $s, s'$ | States |
| $a$ | An action |
| $r$ | A reward |
| $\mathcal{S}$ | Set of all nonterminal states |
| $\mathcal{S}^+$ | Set of all states, incl. terminal state |
| $\mathcal{A}(s)$ | Set of all actions available in state $s$ |
| $\mathcal{R}$ | Set of all possible rewards, a finite subset of $\mathbb{R}$ |
| $\mathcal{C}$ | Subset of (e.g., $\mathcal{R} \subset \mathbb{R}$) |
| $\in$ | Is an element of (e.g. $s \in \mathcal{S}$, $r \in \mathcal{R}$) |
| $\lvert\mathcal{S}\rvert$ | Number of elements in set $\mathcal{S}$ |
| **Time & Policy** |  |
| $t$ | Discrete time step |
| $T, T(t)$ | Final time step of episode, or including $t$ |
| $A_t$ | Action at time $t$ |
| $S_t$ | State at time $t$ |
| $R_t$ | Reward at time $t$ |
| $\pi$ | Policy (decision-making rule) |
| $\pi(s)$ | Action taken in state $s$ under deterministic $\pi$ |
| $\pi(a \mid s)$ | Probability of taking action $a$ in state $s$ under stochastic $\pi$ |
| **Returns** |  |
| $G_t$ | Return following time $t$ |
| $h$ | Horizon (timestep looked up to in forward view) |
| $G_{t:t+n}, G_{t:h}$ | $n$-step return from $t+1$ to $t+n$ or $h$ (discounted/corrected) |
| $G_{t:h}$ | Flat return (undiscounted/unadjusted) from $t+1$ to $h$ |
| $G^\lambda_t$ | $\lambda$-return |
| $G^{\wedge}_t$, $G^{\wedge a}_t$ | Truncated, corrected $\lambda$-return |
| **Transition & Reward Probabilities** |  |
| $p(s', r \mid s, a)$ | Probability of transition to $s'$ with reward $r$ from $s, a$ |
| $p(s' \mid s, a)$ | Probability of transition to $s'$ from $s$ taking $a$ |
| $r(s, a)$ | Expected immediate reward from $s$ after $a$ |
| $r(s, a, s')$ | Expected reward on transition $s \to s'$ under $a$ |
| **Value Functions** |  |
| $v_\pi(s)$ | Value of $s$ under policy $\pi$ (expected return) |
| $v_*(s)$ | Value of $s$ under optimal policy |
| $q_\pi(s, a)$ | Value of taking $a$ in $s$ under $\pi$ |
| $q_*(s, a)$ | Value of taking $a$ in $s$ under optimal policy |
| **Estimators & TD Error** |  |
| $V, V_t$ | Array estimates of $v_\pi$ or $v_*$ |
| $Q, Q_t$ | Array estimates of $q_\pi$ or $q_*$ |
| $\hat{V}(s)$ | Expected approximate action value; e.g. $\hat{V}_t(s) \doteq \sum_a \pi(a|s) Q_t(s, a)$ |
| $U_t$ | Target for estimate at time $t$ |
| $\delta_t$ | Temporal-difference (TD) error at $t$ (a random variable) |
| $\delta_t^s$, $\delta_t^a$ | State- and action-specific forms of TD error |
| $n$ | In $n$-step methods, $n$ is number of steps of bootstrapping |
---

### Function Approximation, Policy Gradient, Advanced Notation

| Symbol | Meaning |
|--------|---------|
| **Weight & Parameter Notation** |  |
| $d$ | Dimensionality—number of components of $\mathbf{w}$ |
| $d'$ | Alternate dimensionality—number of components of $\theta$ |
| $\mathbf{w}, \mathbf{w}_t$ | $d$-vector of weights underlying approximate value function |
| $w_i, w_{t,i}$ | $i$-th component of learnable weight vector |
| $\hat{v}(s, \mathbf{w})$ | Approximate value of $s$ given $\mathbf{w}$ |
| $v_\mathbf{w}(s)$ | Alternate notation for $\hat{v}(s, \mathbf{w})$ |
| $\hat{q}(s, a, \mathbf{w})$ | Approximate value of $(s, a)$ given $\mathbf{w}$ |
| $\nabla \hat{v}(s, \mathbf{w})$ | Column vector of partial derivatives w.r.t. $\mathbf{w}$ |
| $\nabla \hat{q}(s, a, \mathbf{w})$ | Column vector of partial derivatives w.r.t. $\mathbf{w}$ |
| **Feature Vectors & Eligibility Traces** |  |
| $\mathbf{x}(s)$ | Feature vector visible in $s$ |
| $\mathbf{x}(s, a)$ | Feature vector visible in $s$ taking $a$ |
| $x_i(s), x_i(s, a)$ | $i$-th component of $\mathbf{x}(s)$ or $\mathbf{x}(s, a)$ |
| $\mathbf{x}_t$ | Shorthand for $\mathbf{x}(S_t)$ or $\mathbf{x}(S_t, A_t)$ |
| $\mathbf{w}^\top \mathbf{x}$ | Inner product of vectors |
| $\mathbf{v}, \mathbf{v}_t$ | Secondary $d$-vector of weights, used to learn $\mathbf{w}$ |
| $\mathbf{z}_t$ | $d$-vector of eligibility traces at $t$ |
| **Policy Gradient Notation** |  |
| $\theta, \theta_t$ | Parameter vector of target policy |
| $\pi(a \mid s, \theta)$ | Probability of taking $a$ in $s$ given $\theta$ |
| $\pi_\theta$ | Policy corresponding to parameter $\theta$ |
| $\nabla \pi(a \mid s, \theta)$ | Partial derivatives of $\pi(a \mid s, \theta)$ w.r.t. $\theta$ |
| $J(\theta)$ | Performance measure for policy $\pi_\theta$ |
| $\nabla J(\theta)$ | Partial derivatives of $J(\theta)$ w.r.t. $\theta$ |
| $h(s, a, \theta)$ | Preference for $a$ in $s$ based on $\theta$ |
| **Behavior Policy, Baselines, Importance Sampling** |  |
| $b(a \mid s)$ | Behavior policy used to select actions while learning target $\pi$ |
| $b(s)$ | Baseline function $b: \mathcal{S} \to \mathbb{R}$ for policy-gradient methods |
| $b$ | Branching factor for MDP/search tree |
| $\rho_{t:h}$ | Importance sampling ratio for $t$ through $h$ |
| $\rho_t$ | Importance sampling ratio for time $t$ alone, $\rho_t \doteq \rho_{t:t}$ |
| $r(\pi)$ | Average reward (reward rate) for policy $\pi$ |
| $\bar{R}_t$ | Estimate of $r(\pi)$ at time $t$ |
| **State Distributions & Operators** |  |
| $\mu(s)$ | On-policy distribution over states |
| $\mu$ | $\lvert\mathcal{S}\rvert$-vector of the $\mu(s)$ for $s \in \mathcal{S}$ |
| $\|v\|^2_\mu$ | $\mu$-weighted squared norm of $v$, i.e., $\|v\|^2_\mu \doteq \sum_{s \in \mathcal{S}} \mu(s)v(s)^2$ |
| $\eta(s)$ | Expected number of visits to $s$ per episode |
| $\Pi$ | Projection operator for value functions |
| $B_\pi$ | Bellman operator for value functions |

---

### Matrices, Bellman Error, & Error Metrics

| Symbol | Meaning |
|--------|---------|
| **Matrix Notation & Linear Algebra** |  |
| $\mathbf{A}$ | $d \times d$ matrix: $\mathbf{A} \doteq \mathbb{E}\left[ \mathbf{x}_t(\mathbf{x}_t - \gamma \mathbf{x}_{t+1})^\top \right]$ |
| $\mathbf{b}$ | $d$-dimensional vector: $\mathbf{b} \doteq \mathbb{E}[R_{t+1} \mathbf{x}_t]$ |
| $\mathbf{w}_{TD}$ | TD fixed point: $\mathbf{w}_{TD} \doteq \mathbf{A}^{-1}\mathbf{b}$ |
| $\mathbf{I}$ | Identity matrix |
| $\mathbf{P}$ | $\lvert\mathcal{S}\rvert \times \lvert\mathcal{S}\rvert$ matrix of state-transition probabilities under $\pi$ |
| $\mathbf{D}$ | $\lvert\mathcal{S}\rvert \times \lvert\mathcal{S}\rvert$ diagonal matrix with $\mu$ on its diagonal |
| $\mathbf{X}$ | $\lvert\mathcal{S}\rvert \times d$ matrix with the $\mathbf{x}(s)$ as its rows |
| **Bellman Error & Value Error Metrics** |  |
| $\bar{\delta}_\mathbf{w}(s)$ | Bellman error (expected TD error) for $v_\mathbf{w}$ at $s$ |
| $\bar{\delta}_\mathbf{w}$, BE | Bellman error vector (with components $\bar{\delta}_\mathbf{w}(s)$) |
| $\text{VE}(\mathbf{w})$ | Mean square value error: $\text{VE}(\mathbf{w}) \doteq \|v_\mathbf{w} - v_\pi\|^2_\mu$ |
| $\text{BE}(\mathbf{w})$ | Mean square Bellman error: $\text{BE}(\mathbf{w}) \doteq \|\bar{\delta}_\mathbf{w}\|^2_\mu$ |
| $\text{PBE}(\mathbf{w})$ | Mean square projected Bellman error: $\text{PBE}(\mathbf{w}) \doteq \|\Pi \bar{\delta}_\mathbf{w}\|^2_\mu$ |
| $\text{TDE}(\mathbf{w})$ | Mean square temporal-difference error: $\text{TDE}(\mathbf{w}) \doteq \mathbb{E}_b[\rho_t \delta_t^2]$ |
| $\text{RE}(\mathbf{w})$ | Mean square return error |